In [ ]:
#https://stackoverflow.com/questions/73071058/fit-data-with-a-lognormal-function-via-maximum-likelihood-estimators/73112103#73112103
#https://towardsdatascience.com/log-normal-distribution-a-simple-explanation-7605864fb67c

In [ ]:
from functools import partial
import numpy as np
from scipy import optimize, stats

In [ ]:
#Failure probability data from Baker (2015), https://doi.org/10.1193/021113EQS025M
# im = np.array([0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 1])
# number_of_analyses = np.array([40, 40, 40, 40, 40, 40, 40, 40])
# number_of_collapses = np.array([0, 0, 0, 4, 6, 13, 12, 16])

#Failure probability data from Qeshta et al., 2021, https://doi.org/10.1080/15732479.2021.1892774
# im = np.array([4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.46])
# number_of_analyses = np.array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]) 
# number_of_collapses = np.array([0, 0, 0, 0, 0, 0, 1, 3, 7, 9, 10])

#Failure probability data from "twelve_arch_compressive_limit_state_equation.ipynb"
im = np.array([0.289, 0.578, 0.867, 1.156, 1.444, 1.733, 2.022, 2.311, 2.6])
number_of_analyses = np.array([100_000, 100_000, 100_000, 100_000, 100_000, 100_000, 100_000, 100_000, 100_000])
number_of_collapses = np.array([69, 69, 69, 259, 1880, 15286, 74760, 99995, 100_000])

In [ ]:
im_log = np.log(im)

FORMAT_STRING = "{:<20}{:<20}{:<20}"
print(FORMAT_STRING.format("teta", "beta", "log_likelihood_sum"))

In [ ]:
def neg_log_likelihood_sum(params, im_l, no_a, no_c):
    
    teta, beta = params
    theoretical_fragility_function = stats.norm(np.log(teta), beta).cdf(im_l)
    likelihood = stats.binom.pmf(no_c, no_a, theoretical_fragility_function)
    #https://stackoverflow.com/questions/21610198/runtimewarning-divide-by-zero-encountered-in-log
    np.seterr(divide = 'ignore') #equal to IFERROR function of EXCEL
    log_likelihood = np.log(likelihood)
    log_likelihood_sum = np.sum(log_likelihood)

    print(FORMAT_STRING.format(teta, beta, log_likelihood_sum))
    
    return -log_likelihood_sum

In [ ]:
guess_values = (1.6, 0.1) #guess_values = (1.6, 0.1) for Baker and my data, while guess_values = (8, 0.1) for Qeshta et al. (2021) data. 
params_bounds = ((0.01, None), (0.01, None)) #teta and beta must be positive because of the problem physics
desired_tolerance = 1e-6
number_of_iterations = 100

neg_log_likelihood_sum_partial = partial(neg_log_likelihood_sum, im_l=im_log, no_a=number_of_analyses, no_c=number_of_collapses)

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
res = optimize.minimize(neg_log_likelihood_sum_partial, x0 = guess_values, method="Nelder-Mead", options={'maxiter':number_of_iterations, 'disp':True}, bounds = params_bounds, tol = tolerance)
print(res)